In [228]:
from nltk.tokenize import RegexpTokenizer
import pandas
import nltk
import re

In [ ]:
codes = pandas.read_csv("../processed_data/code_top300.csv")
# codes.iloc[:,0] = codes.index.values

In [300]:
identifier = r'[a-zA-Z_][\d\w_]*'

In [229]:
keywords = r'''break|default|func|interface|select|case|defer
                |go|map|struct|chan|else|goto|package|switch
                |const|fallthrough|if|range|type|continue|for|import|return|var'''

In [261]:
# operators and punctuation
operators = r'[\+\-\*\,;\$><!:\.\|&\^=\(\)\[\]\{\}]+'

In [378]:
decimal_literal = r'\d+i?'           
octal_literal = r'0[1-7]*'
hex_literal = r'0[xX][a-fA-F]+'
floating_literal = r''' \d+\.\d*(?:[eE][+-]\d+)?i?     
                | \d+[eE][+-]\d+i?               
                | \.\d+(?:[eE][+-]\d+)?i?        
                '''
string_literal = r'''\"\s*.*\n?\"'''

In [379]:
comments = r'//.*'

In [380]:
patterns =  keywords + '|' + identifier + '|' + operators + '|' \
        + decimal_literal + '|' + octal_literal + '|' + hex_literal + '|' \
        + floating_literal + '|' + string_literal + '|' + comments

In [381]:
r = RegexpTokenizer(patterns)

In [382]:
r.tokenize('print(z12,"    z12 STATE SAVED\n")')

['print', '(', 'z12', ',', '"    z12 STATE SAVED\n"', ')']

In [ ]:
for code in codes['Code']:
    code_lines = list(filter(None, code.split('\n')))
    for line in code_lines[13:]:
        print(line)
        tokens = r.tokenize(line)
        print(tokens)
        ann = []
        for t in tokens:
            if re.match(comments, t):
                ann.append('comment')
            elif re.match(keywords, t):
                ann.append('keyword')
            elif re.match(identifier, t):
                ann.append('identifier')
            elif re.match(operators, t):
                ann.append('operator')
            elif re.match(decimal_literal, t):
                ann.append('decimal_literal')
            elif re.match(octal_literal, t):
                ann.append('octal_literal')
            elif re.match(hex_literal, t):
                ann.append('hex_literal')
            elif re.match(string_literal, t):
                ann.append('string_literal')
            elif re.match(floating_literal, t):
                ann.append('floating_literal')
            else:
                ann.append('undefined')
        print(ann)
        input("================")

        if y == 0 {
['if', 'y', '==', '0', '{']
['keyword', 'identifier', 'operator', 'decimal_literal', 'operator']
            print(z12, "    z12 STATE SAVED\n")
['print', '(', 'z12', ',', '"    z12 STATE SAVED\\n"', ')']
['identifier', 'operator', 'identifier', 'operator', 'string_literal', 'operator']
            print(z13, "    z13 STATE SAVED\n")
['print', '(', 'z13', ',', '"    z13 STATE SAVED\\n"', ')']
['identifier', 'operator', 'identifier', 'operator', 'string_literal', 'operator']
            // Routine 1 is initiator,  it sends 0 to make other process to save the state.
['// Routine 1 is initiator,  it sends 0 to make other process to save the state.']
['comment']
            y = 0
['y', '=', '0']
['identifier', 'operator', 'decimal_literal']
            command12 <- y
['command12', '<-', 'y']
['identifier', 'operator', 'identifier']
            command13 <- y
['command13', '<-', 'y']
['identifier', 'operator', 'identifier']
            // Untill routine 2 and 3 does not 

In [224]:
pattern = r'''(?x)               # set flag to allow verbose regexps
              //
              |".*"|'.*'             # string
              |(?:[A-Z]\.)+         # abbreviations, e.g. U.S.A.
              | \d+(?:\.\d+)?%? # numbers, incl. currency and percentages
              | \w+\.\w+(?:\(\w+\))?       # rand.Intn(100)
              | \w+\s+[<>=]=?\s+\w+   # comparison
              | \w+\+\+     # increament
              | \w+\s*=\s*\S+     # assignment
              | \w+\s*:=\s*\w+     # assignment
              | \w+(?:-\w+)*    # words w/ optional internal hyphens/apostrophe
              | [\(\)+/\-@&*><=]        # special characters with meanings
              
            '''

In [4]:
codes = pandas.read_csv("../processed_data/code_top300.csv")

In [13]:
codes.iloc[:,0] = codes.index.values

In [23]:
list(filter(None, l))

['package main',
 'import "rand"',
 'func Routine1(command12 chan int, response12 chan int, command13 chan int, response13 chan int) {',
 '    // z12 is a variable which stores the value comming from channel 2 and z13 is a variable which stores the value comming from channel 3.',
 '    z12 := 200',
 '    z13 := 200',
 '    m12 := false',
 '    m13 := false',
 '    y := 0',
 '    for i := 0; i < 20; i++ {',
 '        y = rand.Intn(100)',
 "        // If y's value is not 0 then the value will be sent to routine 2 or 3 according to   prime or not.",
 "        // If y's value is 0 then process state (the varibles used by it means z12, z13) and channel state will be saved.[routine 1 is initiator]",
 '        if y == 0 {',
 '            print(z12, "    z12 STATE SAVED\\n")',
 '            print(z13, "    z13 STATE SAVED\\n")',
 '            // Routine 1 is initiator,  it sends 0 to make other process to save the state.',
 '            y = 0',
 '            command12 <- y',
 '            comm